In [1]:
import pandas as pd
import numpy as np
import optuna


import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


import matplotlib.pylab as plt
import seaborn as sns

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
def objective(trial):
#     (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)

    values_df = pd.read_csv('./data/train_values.csv')
    labels_df = pd.read_csv('./data/train_labels.csv')
    data = pd.merge(values_df, labels_df, on='building_id')
    
    # initialize a label encoder
    label_encoder = LabelEncoder()
    #encode categorical columns
    cat_cols = data.select_dtypes('object').columns
    for col in cat_cols:
        data[col] = label_encoder.fit_transform(data[col])
    
    # separate features as X and target as y
    X = data.drop('damage_grade', axis=1)
    data['damage_grade_'] = [x-1 for x in data['damage_grade']]
    y = data['damage_grade_']
    
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)


    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 5, 50),
#         "min_samples_split": trial.suggest_int("min_samples_split", 10, 50),
#         "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 50)
        
    }


    rf = RandomForestClassifier(**param)
    rf.fit(train_x, train_y)
    
    preds = rf.predict(valid_x)
    
    fscore = sklearn.metrics.f1_score(valid_y, preds, average='micro')
    
    return fscore


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-06-20 22:52:21,613] A new study created in memory with name: no-name-3f96471d-90f1-4b76-87fe-0d982032e157
[I 2024-06-20 22:54:09,698] Trial 0 finished with value: 0.7116325473417624 and parameters: {'n_estimators': 368, 'max_depth': 22}. Best is trial 0 with value: 0.7116325473417624.
[I 2024-06-20 22:55:38,694] Trial 1 finished with value: 0.7281518006177932 and parameters: {'n_estimators': 251, 'max_depth': 33}. Best is trial 1 with value: 0.7281518006177932.
[I 2024-06-20 22:56:41,825] Trial 2 finished with value: 0.6874964026016385 and parameters: {'n_estimators': 289, 'max_depth': 15}. Best is trial 1 with value: 0.7281518006177932.
[I 2024-06-20 22:57:23,657] Trial 3 finished with value: 0.7234895723412829 and parameters: {'n_estimators': 111, 'max_depth': 37}. Best is trial 1 with value: 0.7281518006177932.
[I 2024-06-20 22:58:01,996] Trial 4 finished with value: 0.7236430613380403 and parameters: {'n_estimators': 101, 'max_depth': 46}. Best is trial 1 with value: 0.7281

Number of finished trials:  11
Best trial:
  Value: 0.7281518006177932
  Params: 
    n_estimators: 251
    max_depth: 33


In [3]:
best_params = study.best_params

In [4]:
print(best_params)

{'n_estimators': 251, 'max_depth': 33}


In [5]:
values_df = pd.read_csv('./data/train_values.csv')
labels_df = pd.read_csv('./data/train_labels.csv')
data = pd.merge(values_df, labels_df, on='building_id')

# initialize a label encoder
label_encoder = LabelEncoder()
#encode categorical columns
cat_cols = data.select_dtypes('object').columns
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col])

# separate features as X and target as y
X = data.drop('damage_grade', axis=1)
data['damage_grade_'] = [x-1 for x in data['damage_grade']]
y = data['damage_grade_']

train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)


param = best_params


rf = RandomForestClassifier(**best_params)
rf.fit(train_x, train_y)

preds = rf.predict(valid_x)

fscore = sklearn.metrics.f1_score(valid_y, preds, average='micro')

print(fscore)

0.7271733082634638
